In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("./tsdata/modeldata.csv", index_col='Date', parse_dates=True)
df.index.freq = "MS"

df['returns_lag1'] = df['returns'].shift(1)
df['msci_lag1'] = df['msci'].shift(1)
df.dropna(inplace=True)

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)

train = df_scaled[:'2020-01-01']
test = df_scaled['2020-01-02':]

X_train = train[['returns_lag1', 'msci_lag1']].values
y_train = train['returns'].values
X_test = test[['returns_lag1', 'msci_lag1']].values
y_test = test['returns'].values

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

array([[[0.62703835, 0.88188976]],

       [[0.82195416, 0.69291339]],

       [[0.6431248 , 0.71653543]],

       [[0.40964171, 0.68897638]],

       [[0.27021405, 0.79527559]],

       [[0.39004506, 0.44094488]],

       [[0.58216085, 0.64566929]],

       [[0.5379673 , 0.75984252]],

       [[0.48602429, 0.83858268]],

       [[0.18340471, 0.8503937 ]],

       [[0.45647688, 0.68503937]],

       [[0.3227486 , 0.29527559]],

       [[0.14200633, 0.71653543]],

       [[0.41911631, 0.83070866]],

       [[0.40819324, 0.72047244]],

       [[0.35474846, 0.59448819]],

       [[0.46000954, 1.        ]],

       [[0.06013635, 0.66535433]],

       [[0.5522649 , 0.72047244]],

       [[0.59705129, 0.56299213]],

       [[0.39633304, 0.5       ]],

       [[0.67557087, 0.51181102]],

       [[0.54842245, 0.75590551]],

       [[0.36477297, 0.97244094]],

       [[0.15776586, 0.62992126]],

       [[0.52418374, 0.69685039]],

       [[0.42663265, 0.61811024]],

       [[0.56021161, 0.84251

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X_train, y_train, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)


In [ ]:
# Make predictions
predicted = model.predict(X_test)

# Invert scaling if necessary for interpretation of the results
predicted_inv = scaler.inverse_transform(predicted)

rmse = np.sqrt(mean_squared_error(y_test, predicted))
print('Test RMSE: %.3f' % rmse)